In [ ]:
import os
from textwrap import dedent

import duckdb
from gait import FEL, FEL2, Layers
from rich.pretty import pprint

In [ ]:
duckdb_name = os.path.expanduser("~/data/northsea.ddb")

In [ ]:
conn = duckdb.connect(duckdb_name, read_only=True)

_ = conn.execute("INSTALL spatial;LOAD spatial;")

In [ ]:
conn.sql(
    """
select count(*) as "num of pipelines" from pipelines;
"""
)

In [ ]:
conn.sql(
    """
WITH
L as (SELECT DISTINCT p1.OBJECTID
FROM pipelines p1
JOIN pipelines p2
ON 
ST_Intersects(p1.geom, p2.geom)
WHERE
p1.OBJECTID <> p2.OBJECTID)
select pipelines.*
from pipelines
join L
on pipelines.OBJECTID == L.OBJECTID
"""
).df()

In [ ]:
class FEL2SQL:
    @classmethod
    def fel_to_sql(cls, fel2: FEL2) -> str:
        where1 = f" where {fel2.where1}" if fel2.where1 else ""
        where2 = f" where {fel2.where2}" if fel2.where2 else ""
        left_join = False
        st_xxxx = "ST_Intersects"
        # TODO - Convert distance to map spatial reference unit.
        st_dist = ""
        match fel2.relation:
            case "notIntersects":
                left_join = True
            case "withinDistance":
                st_xxxx = "ST_DWithin"
                st_dist = f", {fel2.distance}"
            case "notWithinDistance":
                st_xxxx = "ST_DWithin"
                st_dist = f", {fel2.distance}"
                left_join = "True"
            case "contains":
                st_xxxx = "ST_Contains"
            case "notContains":
                st_xxxx = "ST_Contains"
                left_join = True
            case "within":
                st_xxxx = "ST_Within"
            case "notWithin":
                st_xxxx = "ST_Within"
                left_join = True
            case _:
                pass
        join, where = (
            ("LEFT JOIN", "WHERE R.geom is NULL") if left_join else ("JOIN", "")
        )
        return dedent(
            f"""
        WITH
        L as (SELECT * FROM {fel2.layer1}{where1}),
        R as (SELECT geom FROM {fel2.layer2}{where2})
        SELECT L.* EXCLUDE L.geom, ST_AsWKB(L.geom) SHAPE
        FROM L {join} R
        ON {st_xxxx}(L.geom, R.geom{st_dist})
        {where}
        """
        ).strip()

In [ ]:
layers = Layers.load(os.path.expanduser("~/data/NorthSea.json"))

In [ ]:
fel = FEL(layers.prune_layers(), model="azure/gpt-4o-mini", temperature=0.0)

In [ ]:
fel2 = fel.create_line_2()
pprint(fel2, expand_all=True)

In [ ]:
sql = FEL2SQL.fel_to_sql(fel2.fel)
print(sql)

In [ ]:
conn.sql(sql).df().drop("SHAPE", axis=1)